# Finite-Difference Simulation of Lid-Driven Cavity Flow
**(Final Assignment of Computational Physics 2025)**



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

libertine_installed = True # Only set this to true if the font Linux Libertine is installed

In [4]:
plt.rcParams.update({
    "axes.prop_cycle": plt.cycler(color=["black"] 
                            + plt.rcParams['axes.prop_cycle'].by_key()['color']),
    "font.family": "serif",             # Serif fonts (e.g., Times-like)
    "font.size": 14,                    # Base font size
    "mathtext.fontset": "cm",           # Math Font to match a bit better
    "legend.fontsize": 12,
    "axes.labelsize": 14,
    "axes.titlesize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "figure.dpi": 300,                  # High resolution
    "savefig.dpi": 300,
    "axes.linewidth": 0.8,              # Thin axis lines
    "xtick.direction": "in",            # Ticks pointing inward
    "ytick.direction": "in",
    "xtick.top": True,                  # Ticks on all sides
    "ytick.right": True,
    "xtick.minor.visible": True,
    "ytick.minor.visible": True,
    "grid.linewidth": 0.4
})

if libertine_installed: plt.rcParams.update({"font.serif": ["Linux Libertine"]})  # Same font as in LaTeX)

fsize_small = (4,2.75) # for a single colum in double colum
fsize_large = (6.5,4)

## Building the Functions

In [ ]:
def update_stream_SOR(u, w, omega=1.7, max_iter=10000, tol=1e-5):
    '''
    '''
    N_y, N_x = u.shape() # first y then x, so if we print it out it corresponds to the figure

    for it in range(max_iter):
        max_res = 0.0
        for j in range(1, N_y-1):
            for i in range(1, N_x-1):
                u_new = 0.25 * (u[j+1,i] + u[j-1,i] + u[j,i+1] + u[j,i-1])
                r = u_new - u[j,i]
                u[j,i] += omega * r
                max_r = max(max_res, abs(r))

        if max_res < tol: break

    return u

def update_vorticity(u, w, h, dt, Re):

    N_y, N_x = u.shape() 
    w_new = np.copy(w)

    for j in range(1, N_y-1):
        for i in range(1, N_x-1):
            term1a = (u[j,i+1] - u[j,i-1]) * (w[j+1,i] - w[j-1,i]) 
            term1b = (u[j+1,i] - u[j-1,i]) * (w[j,i+1] - w[j,i-1])
            term2 = (w[j,i+1] + w[j,i-1] + w[j+1,i] + w[j-1,i] - 4*w[j,i])

            w_new[j,i] = w[j,i] + dt/(h*h) (0.25 * (term1a - term1b) + (1/Re) * (term2))
    
    return w_new

def system_evolution(h, dt, Re, T_max, grid_size, omega=1.7, vy_wall=1.0, get_pressure=False):
    '''
    '''
    N_x, N_y = grid_size

    # Initialize fields
    u = np.zeros((N_y, N_x))  # Stream function
    w = np.zeros((N_y, N_x))  # Vorticity
    

    